In [1]:
import pandas as pd
import numpy as np

from datetime import timedelta,datetime
import datetime
from calendar import monthrange
import calendar

from dateutil.relativedelta import relativedelta
import math

from IPython.display import display
import ipywidgets as widgets

import csv

In [2]:
new_data=pd.read_excel("D:\\vikas\\office\\castlight\\standalone\\upload_v1.xlsx")

In [3]:
#existing_allocation_summary=pd.read_csv("D:\\office\\castlight_v3\\generated_summary\\allocation_summary_standalone_1.csv")
#existing_monthly_allocation_raw=pd.read_csv("D:\\office\\castlight_v3\\generated_summary\\month_allocation_summary_standalone_1.csv")

existing_allocation_summary=pd.read_csv("D:\\vikas\\office\\castlight_v3\\generated_summary\\from_standalone\\allocation_summary_standalone_09_06.csv")
existing_monthly_allocation_raw=pd.read_csv("D:\\vikas\\office\\castlight_v3\\generated_summary\\from_standalone\\month_allocation_summary_standalone_09_06.csv")

In [4]:
#filter the data only Line Type as SO
new_data=new_data[new_data['Line Type']=='SO']

In [5]:
 upload_lines = new_data 

In [6]:
retro_upload_line = upload_lines[upload_lines['Allocation Type']=='Retrospective']
upload_lines = retro_upload_line[:2]

In [7]:
# entry for upload date later may be change from data
#upload_date=datetime.datetime(2015, 6, 1).date()

unique_upload_date = upload_lines['Upload Month'].unique()
if len(unique_upload_date) == 1 :
    upload_date = unique_upload_date[0]
else:
    upload_date = unique_upload_date[0]
upload_date

upload_date=datetime.datetime(2015, 6, 1).date()

In [8]:
#Define funtion to get month difference between 2 dates. To be used to calculate allocation term
def get_allocation_term(term,contract_start_date,contract_end_date,allocation_start_date,allocation_end_date):
    return round(term*(abs((allocation_end_date - allocation_start_date).days))/(abs((contract_end_date - contract_start_date).days)),2)

#Define funtion to calculate daily ratable revenue
def get_ratable_revenue(ext_ssp_price,start_date,end_date):
    return ext_ssp_price/(abs((end_date - start_date).days)+1)

#Define funtion to fetch number of days in each month between 2 dates. To be used to calculate monthly revenue table from allocation summary table
# function return month wise days
def get_allocation_durations(start_date,end_date):
    current = start_date
    result = [current]

    current = current.replace(day=1)
    while current <= end_date:
        current += timedelta(days=32)
        current = current.replace(day=1)
        result.append(datetime.datetime(current.year, current.month, 1).date())
    
    durations= []
    for curr in result[:-1]:
        curr_range = calendar.monthrange(curr.year, curr.month)
        curr_duration = (curr_range[1] - curr.day)+1
        if ((curr.month == end_date.month) & (curr.year == end_date.year)):
            durations.append(end_date.day)
        else:
            durations.append(curr_duration)
    return result[:-1],durations


In [9]:
old_allocation =existing_allocation_summary

In [10]:

#Define fintion to perform retrospective allocation 
def perform_retrospective_allocation(new_data,existing_allocation,existing_monthly_allocation=existing_monthly_allocation_raw):
    
    new_data['Contract Start Date']=new_data['Contract Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    new_data['Contract End Date']=new_data['Contract End Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())
    
    #Joining monthly allocation with allocation summary to get month-item level revenue of all past allocations
    df=pd.merge(left=existing_monthly_allocation,right=existing_allocation,left_on=['ra_id','item'],right_on=['Ratable Allocation Id','Item ID']).sort_values('Flag', ascending=False).drop_duplicates(['Sales Order Line ID','month','year'])[['ra_id','item','month','year','revenue','flag','Sales Order Line ID']]

    #fetch the latest allocation summary
    existing_allocation=existing_allocation[existing_allocation['Flag']==existing_allocation['Flag'].max()]

    #Using the latest allocation, append contract start date for each month-item level revenue to 
    df=pd.merge(left=df,right=existing_allocation,how='left',left_on=['Sales Order Line ID'],right_on=['Sales Order Line ID'])[['ra_id','item','month','year','revenue','flag','Ratable Revenue','Contract Start Date','Sales Order Line ID']].sort_values(['item','year','month'])

    df['Contract Start Date']=df['Contract Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())  

    #Get month and year of contract start date
    df['Contract Month']=df['Contract Start Date'].apply(lambda x: x.month)
    df['Contract Year']=df['Contract Start Date'].apply(lambda x: x.year)

    #Fitler the month-item level revenue which are after contract start date and ignoring the history months' revenue which was calculated using old start date
    #Get item level recognized revenue
    recognized_revenue= df[(df['month'] < upload_date.month) & (df['year'] <= upload_date.year) & (df['month'] >= df['Contract Month']) & (df['year'] >= df['Contract Year'])].groupby(['Sales Order Line ID','item']).sum()['revenue']
    recognized_revenue=recognized_revenue.reset_index()
    recognized_months=df[(df['month'] < upload_date.month) & (df['year'] <= upload_date.year) & (df['month'] >= df['Contract Month']) & (df['year'] >= df['Contract Year'])]
    
    recognized_months=recognized_months[['ra_id','item','month','year','revenue','flag']]
    
    recognized_months['flag'] = recognized_months['flag']+1
    
    existing_allocation=existing_allocation.reset_index(drop=True)
    
    #Build allocation id and ratable allocation id
    existing_allocation['Allocation Id'] = existing_allocation.apply(lambda x: str(x['Allocation Id'])[:-3]+ '-' + str('{0:0>2}'.format(int(x['Flag']+2))) ,axis=1 )
    existing_allocation['Ratable Allocation Id'] = existing_allocation.apply(lambda x: str(x['Allocation Id']) + "-" + str('{0:0>2}'.format(x.name +1)), axis = 1)
    
    #Start creating new allocation
    new_allocation=existing_allocation.copy()
    
    #Update the values from new upload line items
    for index, row in existing_allocation.iterrows():
        for i, r in new_data.iterrows():
            if(row['Sales Order Line ID']==r['Orig SO Line ID']):
                new_allocation.loc[new_allocation.index[index],'Contract Start Date']=r['Contract Start Date']
                new_allocation.loc[new_allocation.index[index],'Contract End Date']=r['Contract End Date']
                new_allocation.loc[new_allocation.index[index],'User Count']=r['User Count']
                new_allocation.loc[new_allocation.index[index],'Unit Sell Price']=r['Unit Sell Price']
                new_allocation.loc[new_allocation.index[index],'SSP Term']=r['SSP Term']
                new_allocation.loc[new_allocation.index[index],'Contract Value']=r['Contract Value']
                new_allocation.loc[new_allocation.index[index],'Upload Month']=pd.to_datetime(r['Upload Month'], format='%m/%d/%Y').date()  
        
    
    new_allocation['Contract Start Date']=new_allocation['Contract Start Date'].apply(lambda x : pd.to_datetime(x).date()) 
    new_allocation['Contract End Date']=new_allocation['Contract End Date'].apply(lambda x : pd.to_datetime(x).date())
    #Assign Value to allocation start and end dates
    # new_allocation['Allocation Start Date']=new_allocation['Contract Start Date'].apply(lambda x : max([x,upload_date])) 
    new_allocation['Allocation Start Date']=new_allocation['Contract Start Date']
    new_allocation['Allocation End Date']=new_allocation['Contract End Date']

    #Convert string dates to date format for upcomming calcualtion 
    existing_allocation['Contract Start Date']=existing_allocation['Contract Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    existing_allocation['Contract End Date']=existing_allocation['Contract End Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())
    existing_allocation['Allocation Start Date']=existing_allocation['Allocation Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    existing_allocation['Allocation End Date']=existing_allocation['Allocation End Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())
    existing_allocation['Revenue Start Date']=existing_allocation['Revenue Start Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date()) 
    existing_allocation['Revenue End Date']=existing_allocation['Revenue End Date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d').date())    
    #Calculate allocation term
    new_allocation['Allocation Term']=new_allocation.apply(lambda x: get_allocation_term(x['SSP Term'],x['Contract Start Date'],x['Contract End Date'],x['Allocation Start Date'],x['Allocation End Date']), axis=1)
    
    #Calculate revenue start and ed date
    new_allocation['Revenue Start Date']=new_allocation['Contract Start Date'].apply(lambda x : max([x,upload_date])) 
    new_allocation['Revenue End Date']=new_allocation['Contract End Date'] 

    new_allocation['Reallocation Month']=upload_date
    
    new_allocation['FV Amount']=new_allocation['SSP']*new_allocation['SSP Term']*new_allocation['User Count']
    new_allocation['Allocation Contract Value']=new_allocation['Contract Value']
    new_allocation['EXT SSP Price']=new_allocation['FV Amount']*new_allocation['Allocation Contract Value'].sum()/new_allocation['FV Amount'].sum()
    new_allocation['Flag']+=1
    new_allocation['Ratable Revenue']=new_allocation.apply(lambda x: get_ratable_revenue(x['EXT SSP Price'], x['Contract Start Date'],x['Contract End Date']), axis=1)
    
    new_allocation=new_allocation[existing_allocation.columns]
    
    current_flag=new_allocation['Flag'].unique()[0]
    new_monthly_allocation = pd.DataFrame(columns=['ra_id','item','month','year','revenue','flag'])
    
    for index, row in new_allocation.iterrows():
        months, durations=get_allocation_durations(new_allocation.loc[new_allocation.index[index], 'Revenue Start Date'],new_allocation.loc[new_allocation.index[index], 'Revenue End Date'])
        for idx, val in enumerate(months):
            i=new_monthly_allocation.shape[0]
            new_monthly_allocation.loc[i,'item']=row['Item ID']
            new_monthly_allocation.loc[i,'month']=months[idx].month
            new_monthly_allocation.loc[i,'year']=months[idx].year
            if idx == 0 and abs((upload_date - existing_allocation.loc[existing_allocation.index[index], 'Revenue Start Date']).days)>0:
                update=durations[idx]*row['Ratable Revenue']+((new_allocation.loc[new_allocation.index[index], 'Ratable Revenue'] - existing_allocation.loc[existing_allocation.index[index], 'Ratable Revenue'])*(abs((new_allocation.loc[new_allocation.index[index], 'Revenue Start Date'] - existing_allocation.loc[existing_allocation.index[index], 'Revenue Start Date']).days)))
                new_monthly_allocation.loc[i,'revenue']= update
                
            else:
                new_monthly_allocation.loc[i,'revenue']=durations[idx]*row['Ratable Revenue']

            new_monthly_allocation.loc[i,'ra_id']=row['Ratable Allocation Id']
            new_monthly_allocation.loc[i,'flag']=current_flag
    print(new_monthly_allocation.shape)
    
    
    
    return new_allocation,new_monthly_allocation.append(recognized_months)
    

In [11]:
allocation_id_list = list()

for i in upload_lines['Orig SO Line ID'].unique():
    allocation_id_list.append(existing_allocation_summary[existing_allocation_summary['Sales Order Line ID']==i]['Allocation Id'].values[0])
    
allocation_id_list=set(allocation_id_list)
allocation_id_list


{'21-21-1021-01-01'}

In [12]:
final_new_allocation = pd.DataFrame().reindex_like(existing_allocation_summary)
final_monthly_allocation = pd.DataFrame().reindex_like(existing_monthly_allocation_raw)

In [13]:
for idx,allocation_id in enumerate(allocation_id_list):
#     new_data_subset=new_data[new_data['Allocation ID']==allocation_id]
    new_data_subset=upload_lines
    existing_allocation=existing_allocation_summary[existing_allocation_summary['Allocation Id']==allocation_id]
    final_new_allocation1, final_monthly_allocation1 = perform_retrospective_allocation(new_data_subset,existing_allocation)
    if idx == 0:
        final_monthly_allocation=final_monthly_allocation1
        final_new_allocation=final_new_allocation1
    else:
        final_monthly_allocation=final_monthly_allocation.append(final_monthly_allocation1)
        final_new_allocation=final_new_allocation.append(final_new_allocation1)

c:\users\vikas\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\vikas\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


(44, 6)


In [14]:
list = []
unique_item = old_allocation['Allocation Id'].unique()

for allocation_id in allocation_id_list:
    for item in unique_item:
        if allocation_id not in item:
            list.append(item)
            #print(item)
print(list)

['21-21-1021-02-01', '21-21-1021-03-01']


In [15]:
dataframe= pd.DataFrame()
for item in list:
    df = old_allocation[old_allocation['Allocation Id'] == item]
    df['Flag']= df['Flag']+1
    #print(df)
    dataframe =dataframe.append(df)

#print(dataframe)    
final_new_allocation=final_new_allocation.append(dataframe)

c:\users\vikas\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
list = []
unique_item = existing_monthly_allocation_raw['ra_id'].unique()

for allocation_id in allocation_id_list:
    for item in unique_item:
        if allocation_id not in item:
            list.append(item)
            #print(item)
print(list)

['21-21-1021-02-01-01', '21-21-1021-02-01-02', '21-21-1021-03-01-01', '21-21-1021-03-01-02']


In [17]:
dataframe= pd.DataFrame()
for item in list:
    df = existing_monthly_allocation_raw[existing_monthly_allocation_raw['ra_id'] == item]
    df['flag']= df['flag']+1
    #print(df)
    dataframe =dataframe.append(df)
    
final_monthly_allocation=final_monthly_allocation.append(dataframe)

c:\users\vikas\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
final_new_allocation.to_csv('D:\\vikas\\office\\castlight_v3\\generated_summary\\from_retrospective\\final_retro_allocation1_0915_v3.csv',index=False)
final_monthly_allocation.to_csv('D:\\vikas\\office\\castlight_v3\\generated_summary\\from_retrospective\\final_retro_monthly_allocation1_0915_v3.csv',index=False)

In [15]:
with open('D:\\Master\\Castlight\\Data\\Standalone - TWOC\\allocation_summary1.csv', 'a', newline='') as newFile:
    newFileWriter = csv.writer(newFile)
    for index,row in final_new_allocation.iterrows():
        newFileWriter.writerow(row)

In [16]:
with open('D:\\Master\\Castlight\\Data\\Standalone - TWOC\\monthly_allocation1.csv', 'a', newline='') as newFile:
    newFileWriter = csv.writer(newFile)
    for index,row in final_monthly_allocation.iterrows():
        newFileWriter.writerow(row)